In [8]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check MPS availability
if torch.backends.mps.is_available():
    device = "mps"
    print("✓ Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = "cuda"
    print("✓ Using NVIDIA GPU")
else:
    device = "cpu"
    print("Using CPU")

print(f"avaible device: {device}")

device = "cpu"
print(f"current device: {device}")

✓ Using Apple Silicon GPU (MPS)
avaible device: mps
current device: cpu


In [5]:
print('Downloading Qwen/Qwen2.5-0.5B to your Mac...')
print('This will take 2-5 minutes...')

model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-0.5B',
    dtype=torch.float16,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B')

# Save to local disk
model.save_pretrained('../models/Qwen2.5-0.5B')
tokenizer.save_pretrained('../models/Qwen2.5-0.5B')

print('✓ Model downloaded and saved to ../models/Qwen2.5-0.5B')
print('✓ Ready for local development!')

This will take 2-5 minutes...
✓ Model downloaded and saved to ../models/Qwen2.5-0.5B
✓ Ready for local development!


In [9]:

# Load model on CPU with float32 for better stability
model_path = "../models/Qwen2.5-0.5B"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    # dtype=torch.float16,  # float16 works on MPS!
    dtype=torch.float32,  # Use float32 for CPU
    low_cpu_mem_usage=True,
)
model = model.to(device) 
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [12]:
# Read the r1_zero.prompt from the prompts folder
prompt_path = "prompts/r1_zero.prompt"
with open(prompt_path, "r", encoding="utf-8") as f:
    r1_zero_prompt = f.read()

print("Loaded r1_zero.prompt from prompts folder:")
print(r1_zero_prompt)


Loaded r1_zero.prompt from prompts folder:
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {question}
Assistant: <think>


In [13]:
import json

gsm8k_file = "../data/gsm8k/test.jsonl"
gsm8k_data = []
with open(gsm8k_file, "r", encoding="utf-8") as f:
    for line in f:
        gsm8k_data.append(json.loads(line.strip()))

print(f"Loaded {len(gsm8k_data)} samples from {gsm8k_file}")


Loaded 1319 samples from ../data/gsm8k/test.jsonl


In [14]:
question = gsm8k_data[0]['question']
prompt = r1_zero_prompt.replace("{question}", question)

print('prompt:',prompt)
print('r1_zero_prompt:',r1_zero_prompt)

prompt: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Assistant: <think>
r1_zero_prompt: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think

In [24]:

# Test generation
prompt = "Q: What is 2+2?\nA:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

print("\nGenerating")
import time
start = time.time()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id explicitly
        eos_token_id=tokenizer.eos_token_id   # Also set eos_token_id to be explicit
    )

elapsed = time.time() - start
response = tokenizer.decode(outputs[0])

print(f"Response: {response}")
print(f"Time: {elapsed:.2f}s")
print(f"Tokens/sec: {50/elapsed:.1f}")


Generating
Response: Q: What is 2+2?
A: 4<|endoftext|>
Time: 0.53s
Tokens/sec: 95.2


In [15]:

def get_response(question):
    prompt = r1_zero_prompt.replace("{question}", question)

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        # max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,  # Set pad_token_id explicitly
        eos_token_id=tokenizer.eos_token_id   # Also set eos_token_id to be explicit  
    )
    return tokenizer.decode(outputs[0])

In [29]:
from drgrpo_grader import r1_zero_reward_fn

sample_num = len(gsm8k_data)
eval_num = {}
eval_num['format_only'] = []
eval_num['answer_only'] = []
eval_num['reward'] = []

from tqdm import tqdm

for i in tqdm(range(sample_num), desc="Evaluating"):
    question = gsm8k_data[i]['question']
    import re

    raw_answer = gsm8k_data[i]['answer']
    # Try to parse the number after '####'
    match = re.search(r'####\s*([-+]?\d*\.?\d+)', raw_answer)
    if match:
        answer = match.group(1)
    else:
        # fallback: just use the whole answer if not found
        answer = raw_answer
    print("========Processing sample",i+1,"/",sample_num,"========\n")
    response = get_response(question)
    print('response:',response)
    if "<think>" in response and "</think>" in response:
        think_content = response.split("<think>")[-1].split("</think>")[0].strip()
        print("[think]:", think_content)
    if "<answer>" in response and "</answer>" in response:
        # get the number out of the answer
        # Extract answer content only once
        answer_content = response.split("<answer>")[-1].split("</answer>")[0].strip()
        # Look for comma numbers or generic numbers (matches 120,000, 120000, 123.45, -123)
        match = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\-?\d*\.?\d+)', answer_content)
        if match:
            answer_extracted = match.group(1)
        else:
            answer_extracted = answer_content  # fallback to full content
        print("[answer number]:", answer_extracted)
 
    print('correctanswer:', answer)
    reward = r1_zero_reward_fn(response, answer, fast=True)
    print('reward:',reward)

    if reward['reward'] ==1:
        eval_num['reward'].append(i)
    elif reward['format_reward'] == 1:
        eval_num['format_only'].append(i)
    elif reward['answer_reward'] == 1:
        eval_num['answer_only'].append(i)

print('answer_only:',len(eval_num['answer_only']))
print('format_only:',len(eval_num['format_only']))
print('reward:',len(eval_num['reward']))


Evaluating:   0%|          | 0/1319 [00:00<?, ?it/s]

========Processing sample 1 / 1319 ========



Evaluating:   0%|          | 1/1319 [00:10<3:52:32, 10.59s/it]

response: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Assistant: <think> She eats 3 eggs per day and bakes 4 muffins per day, so she eats 3 + 4 = 7 eggs per day. <think> She sells the remainder at the farmers' market, so she sells 16 - 7 = 9 eggs per day. <think> She makes 2 dollars per fresh duck egg, so she makes 9 * 2 = 18 dollars per day

Evaluating:   0%|          | 2/1319 [00:12<1:55:42,  5.27s/it]

...

response: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: John takes care of 10 dogs.  Each dog takes .5 hours a day to walk and take care of their business.  How many hours a week does he spend taking care of dogs?
Assistant: <think> John takes care of 10 dogs </think> <think> Each dog takes .5 hours a day to walk and take care of their business </think> <think> So, John spends 10 * .5 = 5 hours a day taking care of dogs </think> <think> That means he spends 5 * 7 = 35 hours a week taking care of dogs </think> <answer> John spends 35 hours a week taking care of dogs </answer><|endoftext|>
[think]: That means he spends 5

Evaluating:   3%|▎         | 34/1319 [15:09<9:05:27, 25.47s/it] 

response: A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: Gretchen has 110 coins. There are 30 more gold coins than silver coins. How many gold coins does Gretchen have?
Assistant: <think> She has 110 coins </think> <think> There are 30 more gold coins than silver coins </think> <think> So she has 110 - 30 = 80 silver coins </think> <think> And she has 80 + 30 = 110 gold coins </think> <answer> Gretchen has 110 gold coins </answer><|endoftext|>
[think]: And she has 80 + 30 = 110 gold coins
[answer number]: 110
correctanswer: 70
reward: {'format_reward': 1.0, 'answer_reward': 0.0, 'reward': 0.0}
========Processing sample 

Evaluating:   3%|▎         | 34/1319 [16:43<10:31:54, 29.51s/it]


KeyboardInterrupt: 